# Setup

Este notebook nos va a servir como baseline para el resto de los notebooks sobre los que trabajaremos durante el módulo.

Está pensado como un punto de partida para poder instalar las librerías necesarias para poder trabajar con los notebooks que se irán desarrollando a lo largo del módulo.

In [ ]:
import sys

print(sys.executable)

In [ ]:
!python -m pip install --disable-pip-version-check -q pip --upgrade > /dev/null
!python -m pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null

In [ ]:
!python -m pip install \
    --disable-pip-version-check \
    --quiet \
    --extra-index-url https://download.pytorch.org/whl/cpu \
    awscli \
    awswrangler \
    boto3 \
    botocore \
    tabulate \
    matplotlib==3.5.3 \
    psycopg2-binary==2.9.5 \
    PyAthena==2.14.0 \
    sagemaker-experiments==0.1.39 \
    sagemaker==2.116.0 \
    seaborn==0.12.1 \
    scikit-learn==1.0.2 \
    smdebug==1.0.12 \
    SQLAlchemy==1.4.43 \
    stepfunctions==2.3.0 \
    tensorflow==2.9.2 \
    torch==1.13.0 \
    torch-model-archiver==0.6.0 \
    torchaudio==0.13.0 \
    torchserve==0.6.0 \
    torchvision==0.14.0 \
    transformers==4.24.0

In [ ]:
!conda install --quiet --yes zip

In [ ]:
setup_dependencies_passed = True

### Estableciendo permisos

In [ ]:
import boto3
import botocore.exceptions
import sagemaker
from botocore.config import Config

config = Config(retries={"max_attempts": 10, "mode": "adaptive"})
iam = boto3.client("iam", config=config)

admin = False

role_arn = sagemaker.get_execution_role()
print(f"Role ARN: {role_arn}")

role_name = role_arn.split("/")[-1]
print(f"Role name: {role_name}")

In [ ]:
required_policies = [
    "AdministratorAccess",
    "IAMFullAccess",
#     "SecretsManagerReadWrite",
#     "AmazonS3FullAccess",
#     "AmazonAthenaFullAccess",
#     "ComprehendFullAccess",
#     "AmazonEC2ContainerRegistryFullAccess",
#     "AmazonRedshiftFullAccess",
#     "AWSStepFunctionsFullAccess",
#     "AmazonSageMakerFullAccess",
#     "AmazonKinesisFullAccess",
#     "AmazonKinesisFirehoseFullAccess",
#     "AmazonKinesisAnalyticsFullAccess",
]

Vamos a obtener la diferencia entre los dos set de Python, de tal forma que tengamos una lista de aquellas policies requeridas que no estén actualmente
asociadas a nuestro rol de SageMaker. Para ver este fragmento de código, podemos descomentar cualquiera de las policies de arriba y ejecutar los siguiente

In [ ]:
current_policies = ["AdministratorAccess"]
print(list(set(required_policies).difference(current_policies)))

Es importante tener en cuenta que para poder trabajar con IAM, nuestro rol tiene que tener asociada previamente la policy "IAMFullAccess". Así mismo,
hay que ser conscientes de que esto es un curso, y que en un entorno productivo tenemos que seguir el principio del mínimo privilegio.

In [ ]:
try:
    current_policies = [policy['PolicyName'] for policy in iam.list_attached_role_policies(RoleName=role_name)["AttachedPolicies"]]

    for policy in list(set(required_policies).difference(current_policies)):
        response = iam.attach_role_policy(
            PolicyArn=f"arn:aws:iam::aws:policy/{policy}",RoleName=role_name
        )
        print("Policy {} has been succesfully attached to role: {}".format(policy, role_name))
    setup_iam_roles_passed = True

except botocore.exceptions.ClientError as error:
    print(f"""
        **[ERROR]
        SageMakerExecutionRole needs the AdministratorAccess or IAMFullAccess policy attached.
        **"
    """)
    raise error

except Exception as error:
    raise error

## Guardando variables en el store de Jupyter

Con este paso nos aseguramos de que si alguien ejecuta nuestro notebook, tenga un entorno seguro

[Documentación sobre storemagic](https://ipython.readthedocs.io/en/stable/config/extensions/storemagic.html)

In [ ]:
%store setup_dependencies_passed
%store setup_iam_roles_passed
%store role_arn
%store role_name

In [ ]:
%store

## Liberando recursos AWS

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.b>p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernelbutton>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
script>